### ORD
- ORM(Object Relational Mapping)을 이용하여 RDB와 맵핑해준다.
    - 하나의 관련있는 데이터를 Object로 만들어서 사용하면 
    - 클래스와 연결할 수 있다.
    - 쿼리를 몰라도 사용할 수 있다. 미리 맵핑된 모델이 있기 때문이다.
    - 단점: overhead가 필요하다.
    
- class로 묶어서 object로 관리한다. 
- 알아보기 쉽다.

### SQLAlchemy
- lazy 방식으로 사용. 필요할 때만 연결한다.

In [1]:
!pip install SQLAlchemy==1.3.5

^C


In [1]:
import sqlalchemy

In [2]:
sqlalchemy.__version__

'1.3.5'

In [2]:
from sqlalchemy import create_engine

엔진 생성
- create_engine('sqlite:///foo.db')
- create_engine('sqlite://')
- create_engine('sqlite:///:memory:') -> 메모리에서 사용
- 아직 디비에 접근 하지 않았음. 준비만 했다.

In [4]:
engine = create_engine("sqlite:///:memory:", echo=True)

In [5]:
print(engine)

Engine(sqlite:///:memory:)


In [6]:
type(engine)

sqlalchemy.engine.base.Engine

Create
- Table: class sqlalchemy.schema.Table(\*args, \*\*kw)
- Column: class.sqlalchemy.schema.Column(\*args, \*\*kwargs)
- MetaData : class.sqlalchemy.schema.MetaData
    - 등록된 것을 확인하려면 MetaData.Table로 볼 수 있다.

In [3]:
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey

In [8]:
metadata = MetaData()

In [9]:
users = Table('users', metadata,
              Column('id', Integer, primary_key=True),
              Column('name', String),
              Column('fullname', String)
             )

In [10]:
addresses = Table('addresses', metadata,
                Column('id', Integer, primary_key=True),
                Column('user_id', None, ForeignKey('users.id')),
                Column('email_address', String, nullable=False)
                )

In [11]:
metadata.tables

immutabledict({'users': Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('fullname', String(), table=<users>), schema=None), 'addresses': Table('addresses', MetaData(bind=None), Column('id', Integer(), table=<addresses>, primary_key=True, nullable=False), Column('user_id', Integer(), ForeignKey('users.id'), table=<addresses>), Column('email_address', String(), table=<addresses>, nullable=False), schema=None)})

아직 DB에 생성되지 않았다. 메타데이터에만 등록. 수정 되지 않으므로 싹 날리고 새로 등록 해야한다.

In [12]:
metadata.bind

In [13]:
metadata.create_all(engine)

2019-07-09 11:19:38,984 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-07-09 11:19:38,985 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 11:19:38,987 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-07-09 11:19:38,988 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 11:19:38,991 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2019-07-09 11:19:38,994 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 11:19:38,996 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("addresses")
2019-07-09 11:19:38,997 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 11:19:39,000 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2019-07-09 11:19:39,002 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 11:19:39,004 INFO sqlalchemy.engine.base.Engine COMMIT
2019-07-09 11:19:39,007 INFO sqlalchemy.engine.b

In [15]:
insert = users.insert()

어떤 쿼리인지 알아보기 위해 찍어본다

In [16]:
print(insert)

INSERT INTO users (id, name, fullname) VALUES (:id, :name, :fullname)


In [17]:
insert = users.insert().values(name='kim', fullname='Anonymous, Kim')
print(insert)
insert.compile().params

INSERT INTO users (name, fullname) VALUES (:name, :fullname)


{'name': 'kim', 'fullname': 'Anonymous, Kim'}

db에 연결하기 위해서는 connection해야 한다.
- proxy가 cursor 역할을 한다.
- engine으로 proxy를 얻어온다.

In [26]:
conn = engine.connect()

In [27]:
conn

In [28]:
insert.bind = engine

In [23]:
type(insert)

NameError: name 'insert' is not defined

In [29]:
str(insert)

'INSERT INTO users (name, fullname) VALUES (?, ?)'

In [30]:
result = conn.execute(insert)

2019-07-09 11:31:22,751 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname) VALUES (?, ?)
2019-07-09 11:31:22,753 INFO sqlalchemy.engine.base.Engine ('kim', 'Anonymous, Kim')
2019-07-09 11:31:22,755 INFO sqlalchemy.engine.base.Engine COMMIT


제일 최근에 추가된 primary_key 값을 받아온다.

In [31]:
result.inserted_primary_key

[1]

In [32]:
insert = users.insert()

result = conn.execute(insert, name='Lee', fullname='Unknown, Lee')

result.inserted_primary_key

2019-07-09 11:32:51,463 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname) VALUES (?, ?)
2019-07-09 11:32:51,465 INFO sqlalchemy.engine.base.Engine ('Lee', 'Unknown, Lee')
2019-07-09 11:32:51,466 INFO sqlalchemy.engine.base.Engine COMMIT


[2]

In [33]:
conn = engine.connect()
insert.bind = engine
str(insert)
cur = conn.execute(insert)
cur.inserted_primary_key

2019-07-09 11:36:11,701 INFO sqlalchemy.engine.base.Engine INSERT INTO users DEFAULT VALUES
2019-07-09 11:36:11,702 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 11:36:11,704 INFO sqlalchemy.engine.base.Engine COMMIT


[3]

In [35]:
insert = users.insert()
result = conn.execute(insert,
                     name = 'lee',
                     fullname='Unknown, Lee')
result.inserted_primary_key

2019-07-09 11:37:01,158 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname) VALUES (?, ?)
2019-07-09 11:37:01,160 INFO sqlalchemy.engine.base.Engine ('lee', 'Unknown, Lee')
2019-07-09 11:37:01,162 INFO sqlalchemy.engine.base.Engine COMMIT


[5]

In [36]:
conn.execute(addresses.insert(),[
    {"user_id":1,
    "email_address":"anonymous.kim@test.com"},
    {"user_id":2,
    "email_address":"unknownl.lee@test.com"}
])

2019-07-09 11:38:14,381 INFO sqlalchemy.engine.base.Engine INSERT INTO addresses (user_id, email_address) VALUES (?, ?)
2019-07-09 11:38:14,382 INFO sqlalchemy.engine.base.Engine ((1, 'anonymous.kim@test.com'), (2, 'unknownl.lee@test.com'))
2019-07-09 11:38:14,384 INFO sqlalchemy.engine.base.Engine COMMIT


In [40]:
from sqlalchemy.sql import select

In [42]:
[users]

[Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('fullname', String(), table=<users>), schema=None)]

In [45]:
query = select([users])
result = conn.execute(query)

for row in result:
    print(row)

2019-07-09 11:43:56,273 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2019-07-09 11:43:56,275 INFO sqlalchemy.engine.base.Engine ()
(1, 'kim', 'Anonymous, Kim')
(2, 'Lee', 'Unknown, Lee')
(3, None, None)
(4, 'lee', 'Unknown, Lee')
(5, 'lee', 'Unknown, Lee')


In [50]:
result = conn.execute(select([users.c.name, users.c.fullname]))

for row in result:
    print(row)

2019-07-09 11:46:41,869 INFO sqlalchemy.engine.base.Engine SELECT users.name, users.fullname 
FROM users
2019-07-09 11:46:41,871 INFO sqlalchemy.engine.base.Engine ()
('kim', 'Anonymous, Kim')
('Lee', 'Unknown, Lee')
(None, None)
('lee', 'Unknown, Lee')
('lee', 'Unknown, Lee')


In [52]:
cur = conn.execute(select([users]))
cur.fetchall()

2019-07-09 11:47:45,941 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2019-07-09 11:47:45,942 INFO sqlalchemy.engine.base.Engine ()


[(1, 'kim', 'Anonymous, Kim'),
 (2, 'Lee', 'Unknown, Lee'),
 (3, None, None),
 (4, 'lee', 'Unknown, Lee'),
 (5, 'lee', 'Unknown, Lee')]

In [53]:
from sqlalchemy import and_, or_, not_

In [54]:
print(users.c.id == addresses.c.user_id)

users.id = addresses.user_id


In [55]:
print(users.c.id == 1)

users.id = :id_1


In [56]:
print((users.c.id == 1).compile().params)

{'id_1': 1}


In [57]:
print(or_(users.c.id == 
         addresses.c.user_id, users.c.id == 1))

users.id = addresses.user_id OR users.id = :id_1


In [58]:
print(and_(users.c.id == 
          addresses.c.user_id, users.c.id == 1))

users.id = addresses.user_id AND users.id = :id_1


In [61]:
result = conn.execute(select([users]).where(users.c.id==1))

2019-07-09 13:16:27,273 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users 
WHERE users.id = ?
2019-07-09 13:16:27,275 INFO sqlalchemy.engine.base.Engine (1,)


In [62]:
for row in result:
    print(row)

(1, 'kim', 'Anonymous, Kim')


cross join

In [63]:
result  = conn.execute(select([users, addresses]).where(users.c.id == addresses.c.user_id))

2019-07-09 13:17:32,849 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname, addresses.id, addresses.user_id, addresses.email_address 
FROM users, addresses 
WHERE users.id = addresses.user_id
2019-07-09 13:17:32,851 INFO sqlalchemy.engine.base.Engine ()


In [65]:
for row in result:
    print(row)

(1, 'kim', 'Anonymous, Kim', 1, 1, 'anonymous.kim@test.com')
(2, 'Lee', 'Unknown, Lee', 2, 2, 'unknownl.lee@test.com')


join

In [66]:
from sqlalchemy import join

다음 두 문장은 동일하게 실행된다.
- foreign key를 설정했으므로 on절을 붙이지 않아도 실행 된다.

In [67]:
print(users.join(addresses))

users JOIN addresses ON users.id = addresses.user_id


In [68]:
print(users.join(addresses, users.c.id == addresses.c.user_id))

users JOIN addresses ON users.id = addresses.user_id


In [73]:
query = select([users.c.id, users.c.fullname, addresses.c.email_address]).select_from(users.join(addresses))

In [74]:
result = conn.execute(query).fetchall()

2019-07-09 13:25:47,717 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.fullname, addresses.email_address 
FROM users JOIN addresses ON users.id = addresses.user_id
2019-07-09 13:25:47,719 INFO sqlalchemy.engine.base.Engine ()


In [75]:
for row in result:
    print(row)

(1, 'Anonymous, Kim', 'anonymous.kim@test.com')
(2, 'Unknown, Lee', 'unknownl.lee@test.com')


예제

metadata.clear()

In [2]:
import sqlalchemy

In [2]:
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey, create_engine

In [4]:
engine = create_engine("sqlite:///:memory:", echo=True)

In [7]:
metadata = MetaData()

In [8]:
artist = Table("Artist", metadata,
               Column("id", Integer, primary_key=True),
               Column("name", String, nullable=False),
               extend_existing=True
              )

album = Table("Album", metadata,
            Column("id", Integer, primary_key=True),
             Column("title", String, nullable=False),
              Column("artist_id", Integer, ForeignKey("Artist.id")),
              extend_existing=True
             )

genre = Table("Genre", metadata,
             Column("id", Integer, primary_key=True),
              Column("name", String, nullable=False),
              extend_existing=True
             )

track = Table("Track", metadata,
             Column("id", Integer, primary_key = True),
             Column("title", String, nullable=False),
              Column("length", Integer, nullable=False),
              Column("rating", Integer, nullable=False),
              Column("count", Integer, nullable=False),
              Column("album_id", Integer, ForeignKey("Album.id")),
              Column("genre_id", Integer, ForeignKey("Genre.id")),
              extend_existing=True
             )


engine과 binding해서 db에 메타데이터 생성되어있는 것을 작업해라!

In [9]:
metadata.create_all(engine)

2019-07-09 13:40:32,900 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-07-09 13:40:32,927 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 13:40:32,929 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-07-09 13:40:32,930 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 13:40:32,932 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Artist")
2019-07-09 13:40:32,933 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 13:40:32,935 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Album")
2019-07-09 13:40:32,937 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 13:40:32,938 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Genre")
2019-07-09 13:40:32,939 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 13:40:32,941 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Track")
2019-07-09 13:40:32,942 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 13:40:32,944 INFO sqlalchemy.engin

메모리에 있는 테이블

In [12]:
tables = metadata.tables

In [14]:
for table in tables:
    print(table)

Artist
Album
Genre
Track


데이터베이스에 있는 테이블

In [15]:
for table in engine.table_names():
    print(table)

2019-07-09 13:44:08,462 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2019-07-09 13:44:08,464 INFO sqlalchemy.engine.base.Engine ()
Album
Artist
Genre
Track


engine과 연결

In [17]:
conn = engine.connect()

In [18]:
conn.execute(artist.insert(),[
    {"name":"Led Zepplin"},
    {"name":"AC/DC"}
])

2019-07-09 13:45:22,330 INFO sqlalchemy.engine.base.Engine INSERT INTO "Artist" (name) VALUES (?)
2019-07-09 13:45:22,332 INFO sqlalchemy.engine.base.Engine (('Led Zepplin',), ('AC/DC',))
2019-07-09 13:45:22,334 INFO sqlalchemy.engine.base.Engine COMMIT


In [19]:
conn.execute(album.insert(), [
    {"title":"IV", "artist_id":1},
    {"title":"Who Made Who", "artist_id":2}])

2019-07-09 13:46:34,139 INFO sqlalchemy.engine.base.Engine INSERT INTO "Album" (title, artist_id) VALUES (?, ?)
2019-07-09 13:46:34,141 INFO sqlalchemy.engine.base.Engine (('IV', 1), ('Who Made Who', 2))
2019-07-09 13:46:34,142 INFO sqlalchemy.engine.base.Engine COMMIT


In [21]:
conn.execute(genre.insert(),[
    {"name":"Rock"},
    {"name":"Meetal"}
])

2019-07-09 13:47:09,902 INFO sqlalchemy.engine.base.Engine INSERT INTO "Genre" (name) VALUES (?)
2019-07-09 13:47:09,903 INFO sqlalchemy.engine.base.Engine (('Rock',), ('Meetal',))
2019-07-09 13:47:09,905 INFO sqlalchemy.engine.base.Engine COMMIT


In [22]:
conn.execute(track.insert(),[
    {"title":"Black Dog", "rating":5, "length":297, "count":0, "album_id":1, "genre_id":1},
    {"title":"Stairway", "rating":5, "length":482, "count":0, "album_id":1, "genre_id":1},
    {"title":"About to rock", "rating":5, "length":313, "count":0, "album_id":2, "genre_id":2},
    {"title":"Who Made Who", "rating":5, "length":297, "count":0, "album_id":2, "genre_id":2}
])

2019-07-09 13:49:09,575 INFO sqlalchemy.engine.base.Engine INSERT INTO "Track" (title, length, rating, count, album_id, genre_id) VALUES (?, ?, ?, ?, ?, ?)
2019-07-09 13:49:09,577 INFO sqlalchemy.engine.base.Engine (('Black Dog', 297, 5, 0, 1, 1), ('Stairway', 482, 5, 0, 1, 1), ('About to rock', 313, 5, 0, 2, 2), ('Who Made Who', 297, 5, 0, 2, 2))
2019-07-09 13:49:09,579 INFO sqlalchemy.engine.base.Engine COMMIT


In [25]:
print(track.join(album))

"Track" JOIN "Album" ON "Album".id = "Track".album_id


In [28]:
result = conn.execute(track
                     .select()
                     .select_from(track.join(album))
                     .where(album.c.id==1))

2019-07-09 14:13:07,039 INFO sqlalchemy.engine.base.Engine SELECT "Track".id, "Track".title, "Track".length, "Track".rating, "Track".count, "Track".album_id, "Track".genre_id 
FROM "Track" JOIN "Album" ON "Album".id = "Track".album_id 
WHERE "Album".id = ?
2019-07-09 14:13:07,041 INFO sqlalchemy.engine.base.Engine (1,)


In [29]:
for row in result.fetchall():
    print(row)

(1, 'Black Dog', 297, 5, 0, 1, 1)
(2, 'Stairway', 482, 5, 0, 1, 1)


In [30]:
print(track.join(album))

"Track" JOIN "Album" ON "Album".id = "Track".album_id


In [31]:
print(track.join(album).join(genre))

"Track" JOIN "Album" ON "Album".id = "Track".album_id JOIN "Genre" ON "Genre".id = "Track".genre_id


In [32]:
print(track.join(album).join(artist))

"Track" JOIN "Album" ON "Album".id = "Track".album_id JOIN "Artist" ON "Artist".id = "Album".artist_id


In [33]:
print(track.join(album).join(genre).join(artist))

"Track" JOIN "Album" ON "Album".id = "Track".album_id JOIN "Genre" ON "Genre".id = "Track".genre_id JOIN "Artist" ON "Artist".id = "Album".artist_id


In [41]:
result = conn.execute(track.select([track.c.title, album.c.title, genre.c.name, artist.c.name])
                      .select_from(track.join(album).join(genre).join(artist)))

ArgumentError: SQL expression object expected, got object of type <class 'list'> instead

In [42]:
conn.close()
metadata.clear()

In [43]:
conn

In [44]:
metadata

MetaData(bind=None)

In [45]:
for table in metadata.tables:
    print(table)

In [47]:
for table in engine.table_names():
    print(table)

2019-07-09 14:21:23,428 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2019-07-09 14:21:23,430 INFO sqlalchemy.engine.base.Engine ()
Album
Artist
Genre
Track


In [4]:
engine = create_engine("sqlite:///alchemy_core.db", echo=True)
conn = engine.connect()

metadata = MetaData(bind=engine, reflect=True)
metadata.reflect(bind=engine)

for row in metadata.tables:
    print(row)

2019-07-09 14:43:37,215 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-07-09 14:43:37,235 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 14:43:37,237 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-07-09 14:43:37,238 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 14:43:37,244 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2019-07-09 14:43:37,246 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 14:43:37,250 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2019-07-09 14:43:37,251 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 14:43:37,255 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'users' AND type = 'table'
2019-07-09 14:43:37,257 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 14:43:37,260 INFO sqlalchemy.engine.base.

C:\Anaconda\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SADeprecationWarning: The MetaData.reflect flag is deprecated and will be removed in a future release.   Please use the MetaData.reflect() method.
  after removing the cwd from sys.path.


In [52]:
track = metadata.tables["Track"]
track

KeyError: 'Track'

## base가 metadata의 상위 클래스

In [11]:
from sqlalchemy.ext.declarative import declarative_base

In [6]:
base = declarative_base()

In [7]:
class User(base):
    __tablename__ = "users"
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    password = Column("passwd", String)
    
    def __repr__(self):
        return "<T'User(name='%s', fullname='%s', password='%s')>" % (self.name, self.fullname, self.password)

### Schema

In [8]:
User.__table__

Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('fullname', String(), table=<users>), Column('passwd', String(), table=<users>), schema=None)

In [9]:
User.__mapper__

<Mapper at 0x1384a531b70; User>

### Creative Table

In [10]:
base.metadata.create_all(engine)

2019-07-09 14:43:55,653 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2019-07-09 14:43:55,655 INFO sqlalchemy.engine.base.Engine ()


In [11]:
kim = User(name="kim", fullname="anonymous, Kim", password = "kimbap heaven")

In [12]:
print(kim)

<T'User(name='kim', fullname='anonymous, Kim', password='kimbap heaven')>


In [13]:
print(kim.id)

None


session
- base를 상속 받은 것과 database가 왔다갔다 할 수 있게 하는 애

In [3]:
from sqlalchemy.orm import sessionmaker

In [7]:
Session = sessionmaker(bind = engine)
session = Session()

싱크가 어긋나는 것들이 발생하니 주의하자

In [16]:
session.add(kim)

In [17]:
session.add_all([
    User(name="Lee", fullname="unknown, Lee", password = "123456789a"),
    User(name="Park", fullname="nobody, Park", password = "Parking in Park")
])

In [18]:
session.commit()

2019-07-09 14:46:17,534 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-07-09 14:46:17,538 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, passwd) VALUES (?, ?, ?)
2019-07-09 14:46:17,540 INFO sqlalchemy.engine.base.Engine ('kim', 'anonymous, Kim', 'kimbap heaven')
2019-07-09 14:46:17,559 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, passwd) VALUES (?, ?, ?)
2019-07-09 14:46:17,560 INFO sqlalchemy.engine.base.Engine ('Lee', 'unknown, Lee', '123456789a')
2019-07-09 14:46:17,562 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, passwd) VALUES (?, ?, ?)
2019-07-09 14:46:17,563 INFO sqlalchemy.engine.base.Engine ('Park', 'nobody, Park', 'Parking in Park')
2019-07-09 14:46:17,565 INFO sqlalchemy.engine.base.Engine COMMIT


In [25]:
session.add(User(name="Choi", fullname="amugae, Choi", password="choichoi"))

In [26]:
session.dirty

IdentitySet([])

In [23]:
session.commit()

2019-07-09 14:49:21,524 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-07-09 14:49:21,526 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, passwd) VALUES (?, ?, ?)
2019-07-09 14:49:21,528 INFO sqlalchemy.engine.base.Engine ('Choi', 'amugae, Choi', 'choichoi')
2019-07-09 14:49:21,543 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, passwd) VALUES (?, ?, ?)
2019-07-09 14:49:21,545 INFO sqlalchemy.engine.base.Engine ('Choi', 'amugae, Choi', 'choichoi')
2019-07-09 14:49:21,546 INFO sqlalchemy.engine.base.Engine COMMIT


In [27]:
kim.password="password"

싱크가 맞는지 아닌지 알 수 있다.

In [28]:
session.dirty

2019-07-09 14:50:17,399 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-07-09 14:50:17,401 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname 
FROM users 
WHERE users.id = ?
2019-07-09 14:50:17,403 INFO sqlalchemy.engine.base.Engine (1,)


IdentitySet([<T'User(name='kim', fullname='anonymous, Kim', password='password')>])

In [29]:
session.is_modified(kim)

True

싱크 맞추느라 값을 가져옴?

In [30]:
kim.id

1

In [31]:
for row in session.query(User):
    print(type(row))
    print(row.id, row.name, row.fullname, row.password)

2019-07-09 15:10:45,061 INFO sqlalchemy.engine.base.Engine UPDATE users SET passwd=? WHERE users.id = ?
2019-07-09 15:10:45,062 INFO sqlalchemy.engine.base.Engine ('password', 1)
2019-07-09 15:10:45,084 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, passwd) VALUES (?, ?, ?)
2019-07-09 15:10:45,086 INFO sqlalchemy.engine.base.Engine ('Choi', 'amugae, Choi', 'choichoi')
2019-07-09 15:10:45,088 INFO sqlalchemy.engine.base.Engine SELECT users.passwd AS users_passwd, users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname 
FROM users
2019-07-09 15:10:45,090 INFO sqlalchemy.engine.base.Engine ()
<class '__main__.User'>
1 kim anonymous, Kim password
<class '__main__.User'>
2 Lee unknown, Lee 123456789a
<class '__main__.User'>
3 Park nobody, Park Parking in Park
<class '__main__.User'>
4 Choi amugae, Choi choichoi
<class '__main__.User'>
5 Choi amugae, Choi choichoi
<class '__main__.User'>
6 Choi amugae, Choi choichoi


In [32]:
metadata.clear()

In [33]:
metadata

MetaData(bind=Engine(sqlite:///alchemy_core.db))

In [36]:
Base = declarative_base()

In [37]:
class Artist(Base):
    __tablename__ = "Artist"
    
    id = Column(Integer, primary_key = True)
    name = Column(String)
    
class Album(Base):
    __tablename__="Album"
    
    id = Column(Integer, primary_key = True)
    title = Column(String)
    artist_id = Column(Integer, ForeignKey("Artist.id"))
    
class Genre(Base):
    __tablename__="Genre"
    
    id = Column(Integer, primary_key = True)
    title = Column(String)
    
class Track(Base):
    __tablename__ = "Track"
    
    id = Column("id", Integer, primary_key = True)
    title = Column("title", String, nullable=False)
    length = Column("length", Integer, nullable=False)
    rating = Column("rating", Integer, nullable=False)
    count = Column("count", Integer, nullable=False)
    album_id = Column("album_id", Integer, ForeignKey("Album.id"))
    genre_id = Column("genre_id", Integer, ForeignKey("Genre.id"))

In [39]:
Base.metadata.create_all(engine)

2019-07-09 15:21:23,740 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Artist")
2019-07-09 15:21:23,742 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 15:21:23,746 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Album")
2019-07-09 15:21:23,748 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 15:21:23,750 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Genre")
2019-07-09 15:21:23,752 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 15:21:23,754 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Track")
2019-07-09 15:21:23,755 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 15:21:23,758 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "Artist" (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	PRIMARY KEY (id)
)


2019-07-09 15:21:23,760 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 15:21:23,805 INFO sqlalchemy.engine.base.Engine COMMIT
2019-07-09 15:21:23,808 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "Genre" (
	id INTEGER NOT NULL, 
	title VARCHAR, 
	PRIMARY KE

In [41]:
session.rollback()

C:\Anaconda\Anaconda3\lib\site-packages\sqlalchemy\orm\session.py:552: SAWarning: Session's state has been changed on a non-active transaction - this state will be discarded.
  "Session's state has been changed on "


In [42]:
artist1 = Artist(name = "Led Zepplin")
artist2 = Artist(name = "AC/DC")

session.add_all([artist1, artist2])
session.commit()

2019-07-09 15:21:58,516 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-07-09 15:21:58,518 INFO sqlalchemy.engine.base.Engine INSERT INTO "Artist" (name) VALUES (?)
2019-07-09 15:21:58,519 INFO sqlalchemy.engine.base.Engine ('Led Zepplin',)
2019-07-09 15:21:58,536 INFO sqlalchemy.engine.base.Engine INSERT INTO "Artist" (name) VALUES (?)
2019-07-09 15:21:58,538 INFO sqlalchemy.engine.base.Engine ('AC/DC',)
2019-07-09 15:21:58,540 INFO sqlalchemy.engine.base.Engine COMMIT


In [43]:
album = [Album(title="IV", artist_id=artist1.id),
        Album(title="Who Made Who", artist_id = artist2.id)]

2019-07-09 15:22:48,970 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-07-09 15:22:48,973 INFO sqlalchemy.engine.base.Engine SELECT "Artist".id AS "Artist_id", "Artist".name AS "Artist_name" 
FROM "Artist" 
WHERE "Artist".id = ?
2019-07-09 15:22:48,974 INFO sqlalchemy.engine.base.Engine (1,)
2019-07-09 15:22:48,978 INFO sqlalchemy.engine.base.Engine SELECT "Artist".id AS "Artist_id", "Artist".name AS "Artist_name" 
FROM "Artist" 
WHERE "Artist".id = ?
2019-07-09 15:22:48,980 INFO sqlalchemy.engine.base.Engine (2,)


In [44]:
session.add_all(album)
session.commit()

2019-07-09 15:22:59,313 INFO sqlalchemy.engine.base.Engine INSERT INTO "Album" (title, artist_id) VALUES (?, ?)
2019-07-09 15:22:59,315 INFO sqlalchemy.engine.base.Engine ('IV', 1)
2019-07-09 15:22:59,320 INFO sqlalchemy.engine.base.Engine INSERT INTO "Album" (title, artist_id) VALUES (?, ?)
2019-07-09 15:22:59,322 INFO sqlalchemy.engine.base.Engine ('Who Made Who', 2)
2019-07-09 15:22:59,323 INFO sqlalchemy.engine.base.Engine COMMIT


In [47]:
session.add_all([Genre(title="Rock"), Genre(title="Metal")])
session.commit()

2019-07-09 15:25:55,744 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-07-09 15:25:55,747 INFO sqlalchemy.engine.base.Engine INSERT INTO "Genre" (title) VALUES (?)
2019-07-09 15:25:55,749 INFO sqlalchemy.engine.base.Engine ('Rock',)
2019-07-09 15:25:55,764 INFO sqlalchemy.engine.base.Engine INSERT INTO "Genre" (title) VALUES (?)
2019-07-09 15:25:55,765 INFO sqlalchemy.engine.base.Engine ('Metal',)
2019-07-09 15:25:55,767 INFO sqlalchemy.engine.base.Engine COMMIT


In [5]:
from sqlalchemy.orm import relationship

In [22]:
engine = create_engine("sqlite:///alchemy_core5.db", echo=True)
conn = engine.connect()

metadata = MetaData(bind=engine, reflect=True)
metadata.reflect(bind=engine)

2019-07-09 15:56:07,395 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-07-09 15:56:07,397 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 15:56:07,399 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-07-09 15:56:07,400 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 15:56:07,404 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2019-07-09 15:56:07,406 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 15:56:07,411 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2019-07-09 15:56:07,413 INFO sqlalchemy.engine.base.Engine ()


C:\Anaconda\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SADeprecationWarning: The MetaData.reflect flag is deprecated and will be removed in a future release.   Please use the MetaData.reflect() method.
  after removing the cwd from sys.path.


In [23]:
Base = declarative_base()

In [24]:
class Artist(Base):
    __tablename__ = "Artist"
    
    id = Column(Integer, primary_key = True)
    name = Column(String)
    
    albumList = relationship("Album", back_populates = "artist")
    
class Album(Base):
    __tablename__="Album"
    
    id = Column(Integer, primary_key = True)
    title = Column(String)
    artist_id = Column(Integer, ForeignKey("Artist.id"))
    
    artist = relationship("Artist", back_populates="albumList", uselist=False)
    trackList = relationship("Track", back_populates="album")
    
class Genre(Base):
    __tablename__="Genre"
    
    id = Column(Integer, primary_key = True)
    name = Column(String)
    
    trackList = relationship("Track", back_populates="genre")
    
class Track(Base):
    __tablename__ = "Track"
    
    id = Column("id", Integer, primary_key = True)
    title = Column("title", String, nullable=False)
    length = Column("length", Integer, nullable=False)
    rating = Column("rating", Integer, nullable=False)
    count = Column("count", Integer, nullable=False)
    album_id = Column("album_id", Integer, ForeignKey("Album.id"))
    genre_id = Column("genre_id", Integer, ForeignKey("Genre.id"))
    
    album = relationship("Album", back_populates="trackList", uselist=False)
    genre = relationship("Genre", back_populates="trackList", uselist=False)

# RegularExpression(RE)
- 주어진 텍스트로 부터 패턴이 튀어 나오게끔 하기 위해 사용한다.
- 탐색, replacing, parsing하기 위해 사용할 것이다.
- 텍스트로 처리할 때 훨씬 빠르게 해준다.
- learning curve가 있어서 익숙하게 되기까지 오래걸린다.

### Meta Characters
- . : 모든 문자
- \* : 0번 이상 반복을 의미
- + : 1번 이상 반복을 의미
- {n | min, | min, max} : n번 반복, min 이상 반복, min이상 max이하 반복
- ? : 있거나 말거나

In [25]:
import re

## 패턴 만들기

In [27]:
re.compile("패턴")

re.compile(r'패턴', re.UNICODE)

## 중간에 있는것도 찾는다.

In [28]:
re.search

<function re.search(pattern, string, flags=0)>

## 처음부터 찾는다

In [29]:
re.match()

TypeError: match() missing 2 required positional arguments: 'pattern' and 'string'

## 특정 패턴으로 짜른다?

In [ ]:
re.split()

In [30]:
content = "Hello World"

In [33]:
print(re.search("W", content))

<re.Match object; span=(6, 7), match='W'>


In [34]:
print(re.match("W",content))

None


## 대체, replacing
- 컴파일.sub("pattern", data)

In [35]:
data = """
park 800905-1049118
kim 700905-1059119
"""

In [36]:
pat = re.compile("(\d{6})[-]\d{7}")
print(pat.sub("\g<1>-*******", data)) #\g<1> : 그룹1


park 800905-*******
kim 700905-*******



In [37]:
p = re.compile('(ABC)+')
m = p.search('ABCABC OK?')
print(m.group())

ABCABC


In [38]:
p = re.compile(r'\bclass\b')
print(p.search("no class at all"))
print(p.search("one subclass is"))
print(p.search("the declassified algorithm"))

<re.Match object; span=(3, 8), match='class'>
None
None


In [39]:
p = re.compile(r'\Bclass\B')
print(p.search("no class at all"))
print(p.search("one subclass is"))
print(p.search("the declassified algorithm"))

None
None
<re.Match object; span=(6, 11), match='class'>


In [40]:
p = re.compile(r'\Bclass\b')
print(p.search("no class at all"))
print(p.search("one subclass is"))
print(p.search("the declassified algorithm"))

None
<re.Match object; span=(7, 12), match='class'>
None


In [42]:
m = re.match(r"(\w+) (\w+)", "Isaac Newton, physicist")
print(m.group(0))
print(m.group(1))
print(m.group(2))
print(m.group(1,2))

Isaac Newton
Isaac
Newton
('Isaac', 'Newton')
